<a href="https://colab.research.google.com/github/siddharth1608/datascience/blob/master/recommender_systems/item_based/collaborative_filtering_using_RDDs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommender Systems #

![Recommendations for shopping](https://offerzen.ghost.io/content/images/2018/08/Inner-blog-image.png)
Image courtesy - www.offerzen.com

## Collaborative Filtering - Item based ## 

#### Most popular method behind recommendation systems ####

Recommendation systems are broadly divided into two types
1. Recommend me items based on what my peers have liked in the past - Collaborative filtering
2. Recommend me items based on what I've liked in the past - Content-based filtering

In this notebook, we will implement Collaborative Filtering using Python on Movie Lens Dataset

Let me describe, what would be my approach be like:

*Inputs*
1. User profile: Purchase history of user, preferences
2. Item profile: Characteristics of item. for e.g. movie's genre, lead actor, etc
3. Rating matrix: a table of user's rating for each item. this would be a sparse matrix

*Outputs*


1.   For a given movie, Recommend Top 10 most similar movies


*Steps*:


1.   Form all pairs of movies that a user has rated. Do this for all users. We need this to calculate similarity between every 2 movies
2.   Remove duplicate pairs, for e.g (movie1, movie2) vs (movie2, movie1)
3.   Group all the ratings given to each pair e.g. (movie1, movie2): (3,4), (4,5), (2,4) and so on
4.   For each pair, use the ratings aggregated in previous step to calculate cosine similarity using the following equation:

> > $ sim(itemA, itemB) = \frac{\sum(ratings for  item A)  \sum(ratings for  item B) }{\sqrt(\sum((ratings for item A)^2)) \sqrt(\sum((ratings for item B)^2))}$

> > $ sim(item A, item B) $ falls in the range 0-1, closer to 1 indicates higher similarity




5.  As the final step, for a given test movie, we lookup 10 movies that form a good pair with the given movie with higher similarity scores. We also choose a co-occurence threshold(co-occurence is nothing but the how many users rated for a given pair). By choosing Higher co-occurence threshold, we can claim to be more confident about our rankings.
 








In [0]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import sys
import pprint
np.seed = 24
seed = 24

pp = pprint.PrettyPrinter(depth=4)


In [0]:
# Spark Way

## Uncomment below line to install pyspark
!pip install pyspark
from pyspark.sql import SparkSession



     |████████████████████████████████| 193.9MB 52kB/s 
     |████████████████████████████████| 204kB 44.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/dc/0e/02/e9fdf0bf3ad20284175307d4ab31afcf967604f25f3b4f1d96
Successfully built pyspark


In [0]:
spark = SparkSession \
    .builder \
    .appName("Collaborative filtering - Item based") \
    .getOrCreate()

sc = spark.sparkContext

**Step1** : First let's create a dictionary of all the movie ids with their respective names

In [0]:
movieNames = pd.read_csv('u.item',sep="|", encoding='latin', header=None, index_col=0, usecols=[0,1], names=['id','name'])
movieNamesDict = movieNames.to_dict()
movieNamesDict['name'][51]

We will use the below map function in the preceeding code to convert movies from ids to names. for now assume it will receive a movie pair of ids and return pair with names

In [0]:
def deriveMovieNames(x):
  movieId1 = x[1][0]
  movieId2 = x[1][1]
  
  return (x[0], movieNamesDict['name'][movieId1], movieNamesDict['name'][movieId2])

Time to read the actual ratings file, in the user-item rating world, it is also called as Utility matrix.

In [0]:
reviews_raw = sc.textFile('u.data')

Next we map our data file into structure with user_id as key and (movie, rating) pair as a value

In [0]:
reviews = reviews_raw.map(lambda x: x.split()).map(lambda m: (m[0],(int(m[1]),float(m[2]))))

In [0]:
#reviews.take(2)

[('196', (242, 3.0)), ('186', (302, 3.0))]

For following few lines of statements, our focus will be to form all the pairs of movies that were rated by the same user

In [0]:
reviews_pairs_by_user = reviews.join(reviews)
reviews_pairs_by_user = reviews_pairs_by_user.cache()

In [0]:
reviews_pairs_by_user.take(2)

[('244', ((51, 2.0), (51, 2.0))), ('244', ((51, 2.0), (815, 4.0)))]

As seen, we have duplicate pairs due to self join operation. We will get rid of such pairs by 
only keeping pairs when the absolute value of first movie is less than second

In [0]:
def deDuplicate( item ):
  (user, ratings) = item
  movie1 = ratings[0][0]
  movie2 = ratings[1][0]
  return movie1 < movie2

unique_reviews_pairs_by_user = reviews_pairs_by_user.filter(deDuplicate)
unique_reviews_pairs_by_user = unique_reviews_pairs_by_user.cache()

In [0]:
unique_reviews_pairs_by_user.take(2)

[('244', ((51, 2.0), (815, 4.0))), ('244', ((51, 2.0), (756, 2.0)))]

So far, we have obtained an RDD of type 
> $ user => ((movie1, rating1),(movie2, rating2) $. 

Next, in order to prepare to compute similarity, we will transfrom our RDD into the form

> $ (movie1, movie2) => (rating1, rating2) $

In [0]:
def makeMoviePairAsKey( item ):
  (user, ratings) = item
  movie1 = ratings[0][0]
  movie2 = ratings[1][0]
  rating1 = ratings[0][1]
  rating2 = ratings[1][1]
  
  
  
  
  return ((movie1, movie2), (rating1, rating2))

reviews_by_movies_pairs = unique_reviews_pairs_by_user.map(makeMoviePairAsKey)
reviews_by_movies_pairs = reviews_by_movies_pairs.cache()

In [0]:
reviews_by_movies_pairs.take(2)

[((51, 815), (2.0, 4.0)), ((51, 756), (2.0, 2.0))]

We are pretty close to recommending movies. Next, we will aggregate all the ratings, a movie pair has received. Our RDD will look like this 
> > $(movie1, movie2) =>  [(rating1, rating2), (rating3, rating4), (rating3, rating4)]$

In [0]:
reviews_grpd_by_movies_pairs = reviews_by_movies_pairs.groupByKey()
reviews_grpd_by_movies_pairs = reviews_grpd_by_movies_pairs.cache()

In [0]:
reviews_grpd_by_movies_pairs.take(1)

[((51, 815), <pyspark.resultiterable.ResultIterable at 0x7f57da6eeac8>)]

Great, we are almost der! Next, we will apply cosine similarity over all the ratings a pair has received. Technically this means we will apply a map operation to collection of values per movie pair.

This can be done by using [mapValues()](http://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=mapvalues#pyspark.RDD.mapValues) function of PySpark API

In [0]:
import math
def cosine_similarity(ratingPairs):
  
  sumX2 = 0
  sumY2 = 0
  sumXY = 0
  pairsCount = 0
  for x, y in ratingPairs:
    sumX2 += x * x
    sumY2 += y * y
    sumXY += x * y
    pairsCount += 1
  
  return ( sumXY / (math.sqrt(sumX2) * math.sqrt(sumY2) ),  pairsCount)

In [0]:
moviePairSimilarities = reviews_grpd_by_movies_pairs.mapValues(cosine_similarity).cache()

We are done with the most computation intesive operations, i,e to obtain the similarity matrix. We can save it offline and load at the time of making predictions. 

Pleace uncomment the below cell to save the similarity matrix to disk.

In [0]:
# Save the results if desired 
# moviePairSimilarities.sortByKey() 
# moviePairSimilarities.saveAsTextFile("movie-sims") 

**Show Time !!!** Find the  Top 10 most similar movies

We are using *Three Colors: Red (1994)* as test movie. Let's see which are most similar movies

In [0]:
movie = 60
similarityThreshold = 0.90
coOccurenceThreshold = 20

In [0]:
results = moviePairSimilarities.filter(lambda x: ((x[0][0] == movie) or (x[0][1] == movie)) \
                            and (x[1][0] > similarityThreshold)  and (x[1][1] > coOccurenceThreshold ) 
                            )

In [0]:
pp.pprint(results.map(lambda x: (x[1], x[0]) ).sortByKey(ascending=False).map(deriveMovieNames).take(10))

[((0.9869656221961467, 55),
  'Three Colors: Red (1994)',
  'Three Colors: Blue (1993)'),
 ((0.985960931836453, 31), 'Three Colors: Blue (1993)', 'Henry V (1989)'),
 ((0.982611776170406, 45),
  'Three Colors: Blue (1993)',
  'Three Colors: White (1994)'),
 ((0.9797996983004086, 22),
  'Three Colors: Blue (1993)',
  'Manon of the Spring (Manon des sources) (1986)'),
 ((0.9791633088404823, 21),
  'Three Colors: Blue (1993)',
  'Jean de Florette (1986)'),
 ((0.9782308349399984, 39),
  'Three Colors: Blue (1993)',
  'Nikita (La Femme Nikita) (1990)'),
 ((0.9774049313180511, 22),
  'Three Colors: Blue (1993)',
  'Sex, Lies, and Videotape (1989)'),
 ((0.976513770199478, 28), 'Three Colors: Blue (1993)', 'Big Night (1996)'),
 ((0.9752943944662508, 29), 'Three Colors: Blue (1993)', 'Chinatown (1974)'),
 ((0.9744415217567001, 24), 'Three Colors: Blue (1993)', 'True Romance (1993)')]


Results are not too bad. The first and third recommendations are indeed from the same series. 

Takeaways


1.   Item based is preferred over user based because the product catalog is highly static as compared to the user catalog. Imagine Amazon, Flipkart, where thousands of users are added daily as opposed to few hundreds items.
2.   Similarity item matrix can be generated offline and easily updated with the new ratings
3.   Online computation only is a lookup operation into the similarity matrix. So it is faster to serve in production




Improvements of the model:


1.   Use movie features such as genre, director into training of similarity
2.   Handle cold start users(users who have just started seeing movies) and cold start movies (new movies with very few ratings)

